In [ ]:
from script import df_mobpro_brut, arr_marseille, arr_paris, arr_lyon, contours_comm, transport_dict, contours_comm
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import matplotlib.colors as colors
from scipy.sparse import coo_matrix

In [ ]:
df_mobpro_brut.head()

In [ ]:
# Distribution des modes de transport utilisés pour aller travailler
transport_distribution = df_mobpro_brut['TRANS_LABEL'].value_counts()
print(transport_distribution)

# Visualisation
transport_distribution.plot(kind='bar', color='skyblue')
plt.xticks(rotation=45, ha='right', rotation_mode='anchor')
plt.title('Distribution des modes de transport')
plt.xlabel('Mode de transport')
plt.ylabel('Nombre d’individus')
plt.show()

In [ ]:
# Âge moyen par mode de transport
age_transport = df_mobpro_brut.groupby('TRANS_LABEL')['AGEREVQ'].mean()
print(age_transport)

# Visualisation
age_transport.plot(kind='bar', color='orange')
plt.xticks(rotation=45, ha='right', rotation_mode='anchor')
plt.title('Âge moyen par mode de transport')
plt.xlabel('Mode de transport')
plt.ylabel('Âge moyen')
plt.show()


In [ ]:
# Distribution des modes de transport par catégorie socio-professionnelle avec labels
cs1_transport = pd.crosstab(df_mobpro_brut['CS_LABEL'], df_mobpro_brut['TRANS_LABEL'])

# Heatmap
sns.heatmap(cs1_transport, annot=True, fmt='d', cmap='Blues')
plt.title('Modes de transport par catégorie socio-professionnelle')
plt.xlabel('Mode de transport')

# Inclinaison des labels avec alignement pour que l'extrémité pointe vers l'axe
plt.xticks(rotation=45, ha='right', rotation_mode='anchor')
plt.ylabel('Catégorie socio-professionnelle')
plt.show()


In [ ]:
# Création d'une table croisée dynamique pour compter les déplacements
flux_tot = pd.crosstab(df_mobpro_brut['COMMUNE'], df_mobpro_brut['DCLT'])

# Conversion en DataFrame pour plus de clarté
flux_tot = pd.DataFrame(flux_tot)


# Pour voir les dimensions de la matrice
print("Dimensions de la matrice :", flux_tot.shape)

# Filtrer les données pour chaque groupe de transport
df_trans_45 = df_mobpro_brut[df_mobpro_brut['TRANS'].isin([4, 5])]
df_trans_6 = df_mobpro_brut[df_mobpro_brut['TRANS'] == 6]
df_trans_123 = df_mobpro_brut[df_mobpro_brut['TRANS'].isin([1, 2, 3])]

# Créer les matrices des flux pour chaque groupe
flux_rouge = pd.crosstab(df_trans_45['COMMUNE'], df_trans_45['DCLT'])
flux_jaune = pd.crosstab(df_trans_6['COMMUNE'], df_trans_6['DCLT'])
flux_vert = pd.crosstab(df_trans_123['COMMUNE'], df_trans_123['DCLT'])

# Convertir en DataFrame pour plus de clarté
flux_rouge = pd.DataFrame(flux_rouge)
flux_jaune = pd.DataFrame(flux_jaune)
flux_vert = pd.DataFrame(flux_vert)

# Afficher les dimensions des matrices
print("Dimensions de la matrice (TRANS = 4 ou 5) :", flux_rouge.shape)
print("Dimensions de la matrice (TRANS = 6) :", flux_jaune.shape)
print("Dimensions de la matrice (TRANS = 1, 2 ou 3) :", flux_vert.shape)


In [ ]:
flux_rouge

In [ ]:
# Pour les départs : somme sur l'axe des colonnes (chaque ligne = ville de départ)
df_flux_jaune_depart = flux_jaune.sum(axis=1).reset_index()
df_flux_jaune_depart.columns = ['COMMUNE', 'flux_depart']

# Pour les arrivées : somme sur l'axe des lignes (chaque colonne = ville d'arrivée)
df_flux_jaune_destination = flux_jaune.sum(axis=0).reset_index()
df_flux_jaune_destination.columns = ['DCLT', 'flux_destination']

print("Départ :", df_flux_jaune_depart.head())
print("Destination :", df_flux_jaune_destination.head())

df_flux_vert_depart = flux_vert.sum(axis=1).reset_index()
df_flux_vert_depart.columns = ['COMMUNE', 'flux_depart']

# Pour les arrivées : somme sur l'axe des lignes (chaque colonne = ville d'arrivée)
df_flux_vert_destination = flux_vert.sum(axis=0).reset_index()
df_flux_vert_destination.columns = ['DCLT', 'flux_destination']

print("Départ :", df_flux_vert_depart.head())
print("Destination :", df_flux_vert_destination.head())

df_flux_rouge_depart = flux_rouge.sum(axis=1).reset_index()
df_flux_rouge_depart.columns = ['COMMUNE', 'flux_depart']

# Pour les arrivées : somme sur l'axe des lignes (chaque colonne = ville d'arrivée)
df_flux_rouge_destination = flux_rouge.sum(axis=0).reset_index()
df_flux_rouge_destination.columns = ['DCLT', 'flux_destination']

print("Départ :", df_flux_rouge_depart.head())
print("Destination :", df_flux_rouge_destination.head())


In [33]:
df_flux_jaune_depart_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_jaune_depart,
        left_on='INSEE_COM',
        right_on="COMMUNE",
        how='left')
df_flux_jaune_depart_m["flux_depart"] = df_flux_jaune_depart_m["flux_depart"].fillna(0).astype(int)

df_flux_vert_depart_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_vert_depart,
        left_on='INSEE_COM',
        right_on="COMMUNE",
        how='left')
df_flux_vert_depart_m["flux_depart"] = df_flux_vert_depart_m["flux_depart"].fillna(0).astype(int)

df_flux_rouge_depart_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_rouge_depart,
        left_on='INSEE_COM',
        right_on="COMMUNE",
        how='left')
df_flux_rouge_depart_m["flux_depart"] = df_flux_rouge_depart_m["flux_depart"].fillna(0).astype(int)


df_flux_jaune_destination_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_jaune_destination,
        left_on='INSEE_COM',
        right_on="DCLT",
        how='left')
df_flux_jaune_destination_m["flux_destination"] = df_flux_jaune_destination_m["flux_destination"].fillna(0).astype(int)

df_flux_vert_destination_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_vert_destination,
        left_on='INSEE_COM',
        right_on="DCLT",
        how='left')
df_flux_vert_destination_m["flux_destination"] = df_flux_vert_destination_m["flux_destination"].fillna(0).astype(int)

df_flux_rouge_destination_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_rouge_destination,
        left_on='INSEE_COM',
        right_on="DCLT",
        how='left')
df_flux_rouge_destination_m["flux_destination"] = df_flux_rouge_destination_m["flux_destination"].fillna(0).astype(int)

In [ ]:
plot_flux_gradient(df_flux_rouge_depart_m, "rouge", "Figure - Flux (domicile-travail) de départ \n en voiture/moto par commune","flux_depart")

In [ ]:
plot_flux_gradient(df_flux_jaune_depart_m, "jaune", "Figure - Flux (domicile-travail) de départ \n en transport en commun par commune","flux_depart")

In [ ]:
plot_flux_gradient(df_flux_vert_depart_m, "vert", "Figure - Flux (domicile-travail) de départ \n à pied","flux_depart")

In [ ]:
plot_flux_gradient(df_flux_rouge_destination_m, "rouge", "Figure - Flux (domicile-travail) d'arrivée \n en voiture/moto","flux_destination")

In [ ]:
plot_flux_gradient(df_flux_jaune_destination_m, "jaune", "Figure - Flux (domicile-travail) d'arrivée \n en transport en commun","flux_destination")

In [ ]:
plot_flux_gradient(df_flux_vert_destination_m, "vert", "Figure - Flux (domicile-travail) d'arrivée \n à pied","flux_destination")